In [4]:
# librerias

import warnings
warnings.filterwarnings('ignore')


import pylab as plt
import pandas as pd
import seaborn as sns


from keras.utils import get_file

from keras import Sequential
from keras.layers import Dense

from keras.callbacks import Callback, EarlyStopping

from tensorflow.keras.optimizers.legacy import RMSprop

from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

# Preparación del dataset  
Las columnas tienen los siguientes significados:  
- status (Housing status - a. ready for sale or b. ready to build)
- bed (# of beds)
- bath (# of bathrooms)
- acre_lot (Property / Land size in acres)
- city (city name)
- state (state name)
- zip_code (postal code of the area)
- house_size (house area/size/living space in square feet)
- prev_sold_date (Previously sold date)
- price (Housing price, it is either the current listing price or recently sold price if the house is sold recently)

In [5]:
df_realtor = pd.read_csv('realtor-data.csv')

In [6]:
df_realtor.head()

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,for_sale,145000.0,4.0,2.0,0.10,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,NaN
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [7]:
df_realtor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512159 entries, 0 to 512158
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        512159 non-null  object 
 1   price         512159 non-null  float64
 2   bed           413222 non-null  float64
 3   bath          416941 non-null  float64
 4   acre_lot      407180 non-null  float64
 5   full_address  512159 non-null  object 
 6   street        511543 non-null  object 
 7   city          512100 non-null  object 
 8   state         512159 non-null  object 
 9   zip_code      511962 non-null  float64
 10  house_size    395693 non-null  float64
 11  sold_date     202507 non-null  object 
dtypes: float64(6), object(6)
memory usage: 46.9+ MB


Cambiamos la columna sold_date a formato fecha

In [8]:
df_realtor['sold_date'] = pd.to_datetime(df_realtor['sold_date'], format='%Y-%m-%d', errors='coerce')

In [9]:
df_realtor.sold_date.max()

Timestamp('2022-12-01 00:00:00')

In [ ]:
En las casas no vendidas todavía, cambiamos la fecha por la fecha más reciente, puesto que es el precio al que estaba la casa  
si tenemos en cuenta la fecha más reciente

In [10]:
df_realtor.fillna('2022-12-01 00:00:00')

,status,price,bed,bath,acre_lot,full_address,street,city,state,zip_code,house_size,sold_date
0,for_sale,105000.0,3.0,2.0,0.12,"Sector Yahuecas Titulo # V84, Adjuntas, PR, 00601",Sector Yahuecas Titulo # V84,Adjuntas,Puerto Rico,601.0,920.0,2022-12-01
1,for_sale,80000.0,4.0,2.0,0.08,"Km 78 9 Carr # 135, Adjuntas, PR, 00601",Km 78 9 Carr # 135,Adjuntas,Puerto Rico,601.0,1527.0,2022-12-01
2,for_sale,67000.0,2.0,1.0,0.15,"556G 556-G 16 St, Juana Diaz, PR, 00795",556G 556-G 16 St,Juana Diaz,Puerto Rico,795.0,748.0,2022-12-01
3,for_sale,145000.0,4.0,2.0,0.1,"R5 Comunidad El Paraso Calle De Oro R-5 Ponce,...",R5 Comunidad El Paraso Calle De Oro R-5 Ponce,Ponce,Puerto Rico,731.0,1800.0,2022-12-01
4,for_sale,65000.0,6.0,2.0,0.05,"14 Navarro, Mayaguez, PR, 00680",14 Navarro,Mayaguez,Puerto Rico,680.0,2022-12-01 00:00:00,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...
512154,for_sale,980000.0,3.0,2.0,0.05,"26-25 91st St, East Elmhurst, NY, 11369",26-25 91st St,East Elmhurst,New York,11369.0,1462.0,2001-03-09
512155,for_sale,719000.0,3.0,2.0,2022-12-01 00:00:00,"1111 Banner Ave Apt 6B, Brooklyn, NY, 11235",1111 Banner Ave Apt 6B,Brooklyn,New York,11235.0,1100.0,1998-07-16
512156,for_sale,679000.0,1.0,1.0,2022-12-01 00:00:00,"201 E 83rd St Apt 15E, New York City, NY, 10028",201 E 83rd St Apt 15E,New York City,New York,10028.0,2022-12-01 00:00:00,2020-08-27
512157,for_sale,10875000.0,20.0,22.0,2022-12-01 00:00:00,"189 Woodpoint Rd, Brooklyn, NY, 11211",189 Woodpoint Rd,Brooklyn,New York,11211.0,11335.0,2022-12-01


In [12]:
df_realtor.sold_date.unique()

array([                          'NaT', '2020-02-28T00:00:00.000000000',
       '2019-06-28T00:00:00.000000000', ...,
       '1985-03-15T00:00:00.000000000', '1985-07-11T00:00:00.000000000',
       '1998-07-16T00:00:00.000000000'], dtype='datetime64[ns]')